In [64]:
import pandas as pd
import numpy as np
import os
import geopandas as gd
import openpyxl as px
import psycopg2 as pg
from python_calamine.pandas import pandas_monkeypatch
pandas_monkeypatch()

In [65]:
path_rf = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\Distribuição Regional\mapa RF\Programa Plantio_RF03_divulgado.xlsx'

In [66]:
mapa_rf = pd.read_excel(path_rf,
                             sheet_name='BD',
                             dtype={'Objeto de locação':'object','Id Projeto':'object'},
                             usecols=['Objeto de locação','Projeto', 'Id Projeto', 'RF'])

In [67]:
list_rf = ['RF03','RF04','RF05','RF06']

In [68]:
#Filtrando apenas RF de interesse

mapa_rf = mapa_rf[mapa_rf['RF'].isin(list_rf)]

In [69]:
#Ordenar pelo RF para poder remover duplicatas

mapa_rf = mapa_rf.sort_values('RF', ascending=True)

mapa_rf = mapa_rf.drop_duplicates('Id Projeto')

In [70]:
mapa_rf

,Objeto de locação,Id Projeto,Projeto,RF
1576,0374016,0374,SANTO ANTÔNIO VIII,RF03
1967,0395004,0395,JOAQUIM ÁLVARO,RF03
1969,0396001,0396,JOSÉ ALVARO,RF03
1974,0398001,0398,BOA VISTA VII,RF03
1962,0394011,0394,SÃO JOÃO DO INHEMA,RF03
...,...,...,...,...
780,0224007,0224,SANTA HELENA III,RF06
866,0231319,0231,RIO FEIO,RF06
672,0188301,0188,SÃO JOAQUIM,RF06
918,0241010,0241,MATÃO II,RF06


In [71]:
path_mapa = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\04. Pessoas\Gabriel Moura\01 - Qualidade\23 - Mapas\Projetos\USO_FAZENDA.shp'

In [72]:
projetos = gd.read_file(path_mapa, drive = 'GeoJSON')

projetos = projetos.drop(['DATA_REG','AREA_HA','CD_UNIDADE'],axis=1)

projetos = projetos[projetos['CD_REGIAO'] == 1 ]

#EPS
projetos = projetos.to_crs(epsg = 31983)

projetos = projetos.set_geometry('geometry')

c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver ESRI Shapefile does not support open option DRIVE
  return ogr_read(


In [73]:
projetos = projetos.merge(mapa_rf[['Id Projeto', 'RF']],  left_on= 'ID_PROJETO',  right_on= 'Id Projeto', how='left')

In [74]:
#Removendo itens vázios

projetos = projetos[pd.notna(projetos['RF'])]

In [76]:
save_shp = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\Distribuição Regional\mapa RF\ouput_mapas\rf_mapas.shp"


projetos.to_file(save_shp,
                  driver="ESRI Shapefile",
                  engine='pyogrio')

In [ ]:
#Confirmando se temos todos os itens da base

#projetos.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 132 entries, 0 to 945
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID_PROJETO  132 non-null    object  
 1   NOM_PROJET  132 non-null    object  
 2   CD_REGIAO   132 non-null    float64 
 3   geometry    132 non-null    geometry
 4   Id Projeto  132 non-null    object  
 5   RF          132 non-null    object  
dtypes: float64(1), geometry(1), object(4)
memory usage: 7.2+ KB
